In [ ]:
# Initialize plotting:
%matplotlib inline
# Import all relevant packages
import PIL
from PIL import Image
import scipy.ndimage
import numpy as np, matplotlib.pyplot as plt
import scipy.stats
import scipy as sp
from ipywidgets import interact
from scipy.integrate import solve_ivp

from importlib import reload
from IPython.display import clear_output, display

# Suppress some logging messages in Pillow (Python Imaging Library)
import logging; logging.getLogger('PIL').setLevel(logging.ERROR)  # Suppress PIL messages


: 

# PQ Denoising

$$
  E[u] = \frac{1}{p}\int |\vec{\nabla}u|^p + \lambda \frac{1}{q}\int|u - d|^q,\\
  E'[u] = -\vec{\nabla}u\cdot( |\vec{\nabla}u|^{p-2}\vec{\nabla}u) + \lambda (u-d)|u - d|^{q-2}.
$$

When $p<2$ or $q<2$, the computation of $E'(u)$ can run into numerical issues (divide by zero).  We can regulate these as follows:

$$
  E'[u] = -\vec{\nabla}u\cdot\Bigl((\vec{\nabla}u)(|{\vec{\nabla}u}|^2 + \epsilon)^{(p-2)/2}\Bigr) + \lambda (u-d)\Bigl(|u - d| + \eta\Bigr)^{q-2}.
$$

In [ ]:
import denoise

sigma = 0.4
lam = 0.1
eps = np.finfo(float).eps
eta = np.finfo(float).eps

im = denoise.Image()
d = denoise.Denoise(image=im, sigma=sigma, lam=lam, mode="wrap")

Nx, Ny = im.shape
dx = dy = 1.0
x = (np.arange(Nx) * dx)[:, np.newaxis] #he didn't change these, but he might have meant to 
y = (np.arange(Ny) * dy)[np.newaxis, :]
kx = 2 * np.pi * np.fft.fftfreq(Nx, dx)[:, np.newaxis]
ky = 2 * np.pi * np.fft.fftfreq(Ny, dy)[np.newaxis, :]

fft, ifft = np.fft.fftn, np.fft.ifftn


def gradient(u):
    """Return the gradient of u."""
    ut = fft(u)
    du_dx = ifft(1j * kx * ut).real
    du_dy = ifft(1j * ky * ut).real
    return du_dx, du_dy


def divergence(ux, uy):
    return (ifft(1j * kx * fft(ux) + 1j * ky * fft(uy))).real #alt return (ifft(1j * kx * fft(ux)) + ifft(1j * ky * fft(uy))).real


def E(u, q=2, p=2, d=d.u_noise, lam=lam, eps=eps, eta=eta):
    """Return E.  Note: this is unregulated at q=1"""
    du_dx, du_dy = gradient(u)
    abs_du_p = (du_dx**2 + du_dy**2)**(p / 2)
    return (abs_du_p + lam * abs(u - d)**q).sum() / Nx / Ny


def dE(u, q=2, p=2, d=d.u_noise, lam=lam, eps=eps, eta=eta):
    du_dx, du_dy = gradient(u)
    abs_du_p_m2 = (du_dx**2 + du_dy**2 + eps)**((p - 2) / 2)
    return (-divergence(abs_du_p_m2 * du_dx, abs_du_p_m2 * du_dy) + lam * (u - d) * (abs(u - d) + eta)**(q - 2)) / Nx / Ny #alt eta**2. eps and eta are machine precision terms
            
def pack(u):
    return u.ravel()


def unpack(x):
    return x.reshape((Nx, Ny))


def f(x, **kw):
    u = unpack(x)
    return E(u, **kw)


def df(x, **kw):
    u = unpack(x)
    return pack(dE(u, **kw))


def callback(x, plot=False):
    u = unpack(x)

    msg = f"E={E(u):.4g}"
    if plot:
        import IPython.display

        fig = plt.gcf()
        ax = plt.gca()
        ax.cla()
        IPython.display.clear_output(wait=True)
        im.show(u, ax=ax)
        ax.set(title=msg)
        IPython.display.display(fig)
    else:
        print(msg)

: 

In [ ]:
from functools import partial
from scipy.optimize import minimize

eta = eps = np.finfo(float).eps 
sigma = 0.4 #noise weight
lam = 1 #fidelity weight 
q = 2 
p = 2 

im = denoise.Image()
d = denoise.Denoise(image=im, sigma=sigma, lam=lam, mode="wrap")

x0 = pack(d.u_noise)

args = dict(p=p, q=q, lam=lam, d=d.u_noise, eps=eps, eta=eta)
res = minimize(partial(f, **args),
               x0=x0,
               jac=partial(df, **args),
               method="L-BFGS-B",
               tol=1e-6, #increase tolerance for fast run speed
               callback=partial(callback, plot=False)) #plot  False for faster
clear_output()
u_solve = d.solve()

data = [
    ("Original", d.u_exact),
    (f"Noise σ={sigma}", d.u_noise),
    (f"{q=}, {p=}, {lam=}", unpack(res.x)),
    (f"Solve q=2, p=2, {lam=}", u_solve),
]

fig, axs = denoise.subplots(len(data), height=4)
for ax, (title, u) in zip(axs, data):
    im.show(u, ax=ax)
    ax.set(title=title)

: 

In [ ]:
from functools import partial
from scipy.optimize import minimize

eta = eps = np.finfo(float).eps 
sigma = .4 # noise weight
lam = 1 # fidelity weight 
q = 1.1  # Try 1.1
p = 1.1 # Try 1.1

im = denoise.Image()
d = denoise.Denoise(image=im, sigma=sigma, lam=lam, mode="wrap")

x0 = pack(d.u_noise)

args = dict(p=p, q=q, lam=lam, d=d.u_noise, eps=eps, eta=eta)
res = minimize(partial(f, **args),
               x0=x0,
               jac=partial(df, **args),
               method="L-BFGS-B",
               tol=1e-6, #increase tolerance for fast run speed
               callback=partial(callback, plot=True)) #plot  False for faster got rid of callback.
clear_output()
u_solve = d.solve()

data = [
    ("Original", d.u_exact),
    (f"Noise σ={sigma}", d.u_noise),
    (f"{q=}, {p=}, {lam=}", unpack(res.x)),
    (f"Solve q=2, p=2, {lam=}", u_solve),
]

fig, axs = denoise.subplots(len(data), height=4)
for ax, (title, u) in zip(axs, data):
    im.show(u, ax=ax)
    ax.set(title=title)

: 

: 

: 

: 

: 